In [1]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import pickle
import tools
import sys
import subprocess
from collections import defaultdict
import whisper
module_path = os.path.abspath(os.path.join('./light'))
if module_path not in sys.path:
    sys.path.append(module_path)
from light.ASD import ASD as lightASD
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
inputVideo = "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker092/speaker092_0002.mp4"
#inputVideo = r"inputs/dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
videoFrames = int(videoDuration*25)
fps = 25
#tools.splitVideo(inputVideo,2,videoDuration)

In [6]:
#Carga vídeo
res, facePos, faceFrames = tools.saveMultiFace(inputVideo,detector,50)
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR
print("Nº CARAS:",len(res))
for cara in res.keys():
    print(len(res[cara]))
# cv2.imshow("My Video", res[3][50])
# cv2.waitKey(0)


Nº CARAS: 4
266
266
271
41


In [25]:
model = talkNet()
#model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))
windowSize = 51

# ASD = lightASD()
# ASD.loadParameters("./light/weight/pretrain_AVA_CVPR.model")
# model = ASD.model

07-23 19:06:45 Model para number = 15.01


In [34]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(windowSize,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(0)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=8,num_workers=14) #Cambiar num_workers

2 8


In [42]:
windowSize = 256
datasetTrain = MyDataset(windowSize,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(0)
audioFeature, visualFeature, labels = item
print(len(visualFeature))
audioFeature, visualFeature, labels = audioFeature.cuda().unsqueeze(0), visualFeature.cuda().unsqueeze(0), labels.cuda()
for i in tqdm.tqdm(range(5000)):
    predScores,predLabels = model((audioFeature,visualFeature))

127 508
256


  0%|          | 0/5000 [00:00<?, ?it/s]

  6%|▌         | 292/5000 [00:05<01:29, 52.90it/s]


KeyboardInterrupt: 

In [27]:
correctFrames, totalFrames = 0, 0
correctSamples, totalSamples = 0, 0
totalPreds = []
totalScores = []
for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
    with torch.no_grad():    
        audioFeature, visualFeature, labels = audioFeature.cuda(), visualFeature.cuda(), labels.cuda()
        predScores,predLabels = model((audioFeature,visualFeature)) #(audioFeature,visualFeature) si talknet
        batchPreds = torch.reshape(predLabels, labels.shape)
        #Precision a nivel de video
        videoPreds = torch.mode(batchPreds,dim=1)[0]
        totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
        for ten in torch.split(predScores[:,1],windowSize):
            totalScores.append(torch.mean(ten).detach().cpu().numpy())
        labelMode = torch.mode(labels,dim=1)[0]
        correctSamples += (videoPreds == labelMode).sum().float()
        totalSamples += len(labels)
        # Precision a nivel de frame
        labels = labels.reshape((-1))
        correctFrames += (predLabels == labels).sum().float()
        totalFrames += len(labels) 
#7865
print("Frame acc:", correctFrames/totalFrames)
print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res[0])).unsqueeze(0)
iVideo = tools.padVideo(res[0],center,51).unsqueeze(0)
print(iAudio[40],item[0][40])
print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
print(audio[0],item[0][0])

  0%|          | 0/3751 [00:00<?, ?it/s]

100%|██████████| 3751/3751 [02:01<00:00, 30.84it/s]


Frame acc: tensor(0.7634, device='cuda:0')
Sample acc: tensor(0.7638, device='cuda:0')


IndexError: index 40 is out of bounds for dimension 0 with size 1

In [14]:
totalScores = defaultdict(list)
sideWindowSize = int((windowSize-1)/2)
print("sidewindow:",sideWindowSize)
sequential = True
meanWSize = 11
meanSideSize = int((meanWSize-1)/2)
thr = 0.190

sidewindow: 10


In [15]:
if sequential:
    # SECUENCIAL
    for actualSpeaker in res.keys():
        print("SPEAKER", actualSpeaker)
        for i in range(0,len(res[actualSpeaker])+windowSize+1,windowSize):
            #center = i
            center = faceFrames[actualSpeaker][0]+i
            #print("FRAME Nº ",center)
            iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
            iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
            #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
            scores,labels= model((iAudio,iVideo))
            totalScores[actualSpeaker].extend(scores[:,1].detach().cpu().numpy().tolist())
            #predArray[actualSpeaker].extend(labels.detach().cpu().numpy().tolist())

        # MEAN SLIDING WINDOW
        for i in range(len(totalScores[actualSpeaker])):
            ini = max(0,i-meanSideSize) # No negative values
            end = i+meanSideSize+1 # +1 as python does not take into account last value
            #print(ini,end)
            totalScores[actualSpeaker][i] = np.mean(totalScores[actualSpeaker][ini:end])
        print(totalScores[actualSpeaker])
else:
    # MINIMO
    for actualSpeaker in res.keys():
        print("SPEAKER", actualSpeaker, "LENGTH:", len(res[actualSpeaker]))
        for i in range(len(res[actualSpeaker])):
            #center = i
            center = faceFrames[actualSpeaker][0]+i
            print("FRAME Nº ",center)
            iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
            iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
            #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
            scores,labels= model((iAudio,iVideo))
            print("\tProb Pos ->",scores[sideWindowSize][1])
            totalScores[actualSpeaker].append(scores[sideWindowSize][1].detach().cpu().numpy())
        
        # MEAN SLIDING WINDOW
        for i in range(len(totalScores[actualSpeaker])):
            ini = max(0,i-meanSideSize) # No negative values
            end = i+meanSideSize+1 # +1 as python does not take into account last value
            #print(ini,end)
            totalScores[actualSpeaker][i] = np.mean(totalScores[actualSpeaker][ini:end])
        print(totalScores[actualSpeaker])



NameError: name 'res' is not defined

In [105]:
# SMOOTHING
predArray = defaultdict(list)

# APPLY THRESHOLD
for actualSpeaker in res.keys():
    # Delete < 0 frame predictions (initial)
    #Solo para metodo secuencial ->
    if sequential:
        totalScores[actualSpeaker] = totalScores[actualSpeaker][sideWindowSize:len(res[actualSpeaker])+sideWindowSize]
    # Classify as speaking if score higher than threshold
    for sc in totalScores[actualSpeaker]:
        predLabel = 1 if sc > thr else 0
        predArray[actualSpeaker].append(predLabel)

    print(len(predArray[actualSpeaker]))

266
266
271
41


In [106]:
def createVideo(videoName,imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/videos/{videoName}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

# Returns a list of [start,end] timestamps in seconds of the parts where a speaker is speaking with minimum length of minLength frames
def getSpeaking(arr,minLength,fps):
    prev_idx = 0
    posFrames = 0
    idx_list = []
    for i,num in enumerate(arr):
        if num == 1:
            posFrames +=1
        else:
            if i-prev_idx >= minLength:
                idx_list.append((prev_idx/fps,i/fps))
            prev_idx = i
            posFrames = 0
    return idx_list

def saveFullVideo(videoName):
    cap = cv2.VideoCapture(inputVideo)
    videoImages = []
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while True:
        ret, image = cap.read()
        videoImages.append(image)
        if ret == False:
            break


    for speakerN in totalScores.keys():
        for i, fr in enumerate(faceFrames[speakerN]): #Para cada frame en el que aparezca la cara
            try:
                image = videoImages[fr]
                greenValue = int(255*predArray[speakerN][i])
                redValue = 255-greenValue
                color = (0,greenValue,redValue)
                #print(color)
                xmin,ymin,xmax,ymax = facePos[speakerN][i]
                image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
                cv2.putText(image, "{:.2f}".format(totalScores[speakerN][i]*100), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
            except:
                pass
    cap.release()
    cv2.destroyAllWindows()
    
    
    createVideo(videoName,videoImages,audioPath,width,height)

In [107]:
#Asignación de bounding boxes y probabilidades al video de output
os.makedirs("outputs/videos", exist_ok=True)
path = os.path.normpath(inputVideo)
videoName = str(path.split(os.sep)[-1])
saveFullVideo(videoName)

#print(predArray[3])

b''


In [108]:
# GUARDAR DATOS 
# COGER SOLO FRAGMENTO DE AUDIO EN EL QUE SE DETECTA HABLANTE
os.makedirs("outputs/pickles", exist_ok=True)
model = whisper.load_model("small")
transcription = model.transcribe(audioPath, language="es", verbose=True,word_timestamps=True)
pkDict = {"facePos":facePos,"faceFrames":faceFrames,"preds":predArray,"transcription":transcription}
with open("outputs/npz/"+videoName+'.pkl', 'wb') as f:  # open a text file
    pickle.dump(pkDict, f) # serialize the list

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:00.000 --> 00:08.340]  es que había una estrategia diseñada de realizar un golpe de Estado institucional al margen de la legalidad.
[00:08.500 --> 00:10.140]  Y alguien deberá responder por ello.


In [ ]:
# EXTRAER PARTE APROPIADA DE LA TRANSCRIPCIÓN
wordArr = []
alignArr = []
for seg in transcription["segments"]:
    for w in seg["words"]:
        wordArr.append(w["word"])
        alignArr.append((w["start"],w["end"]))
print(wordArr)
print(alignArr)
pdRows = []

for speakerN in totalScores.keys():
    for (ini,end) in getSpeaking(predArray[speakerN],5,25):
        iniW, endW = -1, -1
        for i, (alignIni, alignEnd) in enumerate(alignArr):
            if iniW == -1 and ini <= alignEnd and ini >= alignIni:
                iniW = i
            if endW == -1 and end <= alignEnd and end >= alignIni:
                endW = i
        if iniW > -1 and endW > -1:
            newRow = {'video':inputVideo, 'speaker':speakerN, 'ini': ini, 'end':end, 'dataPath': "outputs/npz/"+videoName+".pkl", 'transcription':''.join(wordArr[iniW:endW+1])}
            pdRows.append(newRow)

df = pd.DataFrame(pdRows,columns=['video', 'speaker', 'ini', 'end', 'dataPath', 'transcription'])
df.to_csv(r"outputs/res.csv")

In [ ]:
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
df = pd.read_csv("testSamples.csv")
df2 = pd.DataFrame({'pred':totalPreds})
df3 = pd.DataFrame({'posScore':totalScores})
df["pred"] = df2
df["posScore"] = df3
df.to_csv("testPredsX.csv")
g_truth = pd.read_csv("testSamples.csv")["label"].values.tolist()
fpr, tpr, thresholds = metrics.roc_curve(g_truth, totalScores)
auc = metrics.roc_auc_score(g_truth, totalScores)
np.savetxt('cli.txt', df[df["label"]==1]["posScore"].values, fmt='%f')
np.savetxt('imp.txt', df[df["label"]==0]["posScore"].values, fmt='%f')
nCorrect = 0
for i in df[df["label"]==1]["posScore"].values:
    if i >=0.5:
        nCorrect +=1

In [ ]:
def to_labels(pos_probs, threshold):
 return (pos_probs >= threshold).astype('int')

thresholds = np.arange(0, 1, 0.001)
probs = df["posScore"].values.tolist()
test_y = df["label"].values.tolist()

scores = [f1_score(test_y, to_labels(probs, t)) for t in thresholds]
ix = np.argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
plt.plot(fpr,tpr)